# 图片操作

In [24]:
import numpy as np
from scipy.ndimage import grey_dilation,grey_erosion,grey_opening,grey_closing
import arcpy
from arcpy import env
import os

In [16]:
data_path = r'D:\ArcgisData\pred_tl\pred_database\TL_basedata.gdb\TL_pred_raster_C'

In [3]:

# 加载栅格数据
raster_data = arcpy.RasterToNumPyArray(data_path)

In [25]:
# 定义膨胀操作
def dilation_operation(data):
    return grey_dilation(data, size=(3, 3))

# 定义腐蚀操作
def erosion_operation(data):
    return grey_erosion(data, size=(3, 3))
# 定义开操作
def open_operation(data):
    return erosion_operation(dilation_operation(data))
# 定义闭操作
def close_operation(data):
    return dilation_operation(erosion_operation(data))
# 定义边缘检测
def edge_detection(data):
    return data - erosion_operation(data)
# 定义灰度形态学梯度
def gradient(data):
    return dilation_operation(data) - erosion_operation(data)
# 定义顶帽操作
def top_hat(data):
    return data - open_operation(data)
# 定义底帽操作
def bottom_hat(data):
    return close_operation(data) - data
# 定义灰度形态学开操作
def grey_open(data):
    return grey_erosion(data) - grey_dilation(data)
# 定义灰度形态学闭操作
def grey_close(data):
    return grey_dilation(data) - grey_erosion(data)
# 定义灰度形态学梯度
def grey_gradient(data):
    return grey_dilation(data) - grey_erosion(data)
# 定义灰度形态学顶帽操作
def grey_top_hat(data):
    return data - grey_open(data)
# 定义灰度形态学底帽操作
def grey_bottom_hat(data):
    return grey_close(data) - data
# 定义灰度形态学边缘检测
def grey_edge_detection(data):
    return grey_dilation(data) - grey_erosion(data)



# 转栅格
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster

In [10]:
# 进行膨胀操作
dilated_data = dilation_operation(raster_data)

In [26]:

# 进行腐蚀操作
eroded_data = erosion_operation(raster_data)

In [17]:
# 工作空间
env.workspace = r"D:\ArcgisData\basedata\basetrain_30m.gdb"
env.extent = "DEM"

In [21]:

# 膨胀后数据按掩膜处理
result_path = r"D:\ArcgisData\pred_tl\pred_database\TL_basedata.gdb"
result_raster = mask_raster(dilated_data,'DEM', 30)
result_raster.save(os.path.join(result_path,"TL_dilated1"))

In [27]:
# 腐蚀后数据按掩膜处理
result_path = r"D:\ArcgisData\pred_tl\pred_database\TL_basedata.gdb"
result_raster = mask_raster(eroded_data,'DEM', 30)
result_raster.save(os.path.join(result_path,"TL_eroded2"))

In [51]:
# 创建Raster对象
raster = arcpy.sa.Raster(data_path)
int_raster = arcpy.sa.Int(raster)
zone_values = 1

# 应用膨胀操作来填充小间隙
dilated_raster = arcpy.sa.Expand(int_raster,  zone_values,[0,1,2,3,4,5,6,7,8,9,10],'MORPHOLOGICAL')

# 应用腐蚀操作来去除小的杂散区域
eroded_raster = arcpy.sa.Shrink(dilated_raster, zone_values,[0,1,2,3,4,5,6,7,8,9,10], 'MORPHOLOGICAL')
float_raster = arcpy.sa.Float(eroded_raster)



In [54]:
# 腐蚀后数据按掩膜处理
result_path = r"D:\ArcgisData\pred_tl\pred_database\TL_basedata.gdb"
arcpy.management.CopyRaster(float_raster, os.path.join(result_path, "float_raster"))

<Result 'D:\\ArcgisData\\pred_tl\\pred_database\\TL_basedata.gdb\\float_raster'>